In [2]:
import numpy as np
import pandas as pd
import sklearn.naive_bayes
import sklearn.linear_model
import sklearn.metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from xgboost import XGBClassifier

import seaborn as sns
import matplotlib.pyplot as plt
import textstat


In [8]:


def fit_model(X_train, y_train, X_test, y_test):

    # classifier = XGBClassifier(objective='multi:softmax', max_depth=4, learning_rate=.8, n_estimators=100)
    # classifier = make_pipeline(StandardScaler(), SGDClassifier(max_iter=1000, tol=1e-3))
    # classifier = GradientBoostingClassifier(n_estimators=100, learning_rate=0.8, max_depth=1, random_state=0)
    # classifier = RandomForestClassifier(max_depth=5, random_state=0)
    # classifier = sklearn.linear_model.LogisticRegression(multi_class='multinomial', random_state=0, max_iter=3000)
    classifier = sklearn.naive_bayes.GaussianNB()

    classifier.fit(X_train, y_train)

    y_pred = classifier.predict(X_test)

    matrix = sklearn.metrics.confusion_matrix(y_test, y_pred)

    return matrix

def get_data(filename):
    dataframe = pd.read_excel(filename)
    dataframe['level'] = dataframe['level'].replace('low', 0).replace('medium', 1).replace('high', 2)
    X = dataframe[features]
    y = dataframe['level']
    return X,y,dataframe
    
    
def run_models(filename, features, num_folds):
    X, y,dataframe = get_data(filename)
    folds = StratifiedKFold(num_folds, shuffle=True, random_state=0)

    model_predictions = {}

    for i, (train_index, test_index) in enumerate(folds.split(X, y)):
        predictions = fit_model(X.loc[train_index], y.loc[train_index], X.loc[test_index], y.loc[test_index])
        model_predictions[i] = predictions

    total_matrix = model_predictions[0]

    for i in range(1, num_folds):
        total_matrix += model_predictions[i]
    print(model_predictions)
    print(total_matrix)
    accuracy = (total_matrix[0][0] + total_matrix[1][1] + total_matrix[2][2]) / 79
    print(features)
    print(accuracy)
    print(total_matrix)



# Full features list: ['count', 'cont', 'retain', 'shift', 'one_mention',
#        'multi_mention', 'flesch_reading_ease', 'difficult words', 'dale_chall_readability_score',
#        'textstat difficult words', 'arc_length', 'average_height', 'height',
#        'num_edus']

features = ['count', 'cont', 'retain', 'shift', 'one_mention', 'multi_mention', 'flesch_reading_ease']
num_folds = 5
filename ='data/TOEFL Annotation_mine.xlsx'

run_models(filename, features, num_folds)

{0: array([[19,  4,  2],
       [ 5, 14,  6],
       [ 2,  6, 21]]), 1: array([[4, 1, 0],
       [1, 3, 1],
       [1, 1, 4]]), 2: array([[3, 1, 1],
       [2, 3, 0],
       [0, 1, 5]]), 3: array([[4, 1, 0],
       [0, 2, 3],
       [0, 0, 6]]), 4: array([[5, 0, 0],
       [2, 2, 1],
       [1, 3, 1]])}
[[19  4  2]
 [ 5 14  6]
 [ 2  6 21]]
['count', 'cont', 'retain', 'shift', 'one_mention', 'multi_mention', 'flesch_reading_ease']
0.6835443037974683
[[19  4  2]
 [ 5 14  6]
 [ 2  6 21]]


In [24]:
from scipy.special import softmax

dropped_features = [ "flesch_reading_ease","average_height","height","num_edus","num_rel","prop_rel"]
def feature_engineering(filename):
    X, y,dataframe = get_data(filename)
    dataframe = dataframe.drop(["Unnamed: 0","name", "flesch_reading_ease","average_height","height","num_edus","num_rel","prop_rel"],axis=1)
    corr = dataframe.corr()
    result = softmax(corr["level"])
    print(corr["level"])
    print(result)

feature_engineering(filename)


level                           1.000000
count                           0.664785
cont                            0.157801
retain                          0.186319
shift                           0.392407
one_mention                     0.568834
multi_mention                   0.568787
difficult words                 0.753427
dale_chall_readability_score    0.363783
arc_length                      0.016346
Name: level, dtype: float64
[0.16345187 0.11689816 0.07040887 0.07244575 0.0890257  0.10620297
 0.10619807 0.12773349 0.08651356 0.06112156]
